In [113]:
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

import keras.backend as K

In [114]:
batch_size = 64
img_height = 224
img_width = 224
num_classes = 4

In [115]:
image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

def preprocces_image(example_proto):
    features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.io.decode_raw(features['image'], tf.uint8)
    image.set_shape([img_width*img_width])
    image = tf.reshape(image, (img_width, img_width, 1))
    image = tf.image.grayscale_to_rgb(image)
    
    label = random.randint(0,3)    
    image = tf.image.rot90(image, label)
    
    label = tf.one_hot(label, 4)

    
    return image, label

def read_dataset(batch_size, filenames, repeat=True):
    dataset = tf.data.TFRecordDataset(filenames)

    dataset = dataset.map(preprocces_image, num_parallel_calls=10)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    if repeat:
        dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=10 * batch_size, seed = 42)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [116]:
tfrecord_files = sorted(os.listdir("AffectNet_Manually_Annotated_Images_224x224_tfrecords"))
tfrecord_paths = [os.path.join("AffectNet_Manually_Annotated_Images_224x224_tfrecords", file) for file in tfrecord_files]

num_shards_for_val_set = 10
num_of_imgs_per_shard = 3500
total_num_of_imgs = 337384

val_ds_num_of_imgs = num_of_imgs_per_shard*num_shards_for_val_set
train_ds_num_of_imgs = total_num_of_imgs - val_ds_num_of_imgs

train_tfrecord_paths = tfrecord_paths[num_shards_for_val_set:]
val_tfrecord_paths = tfrecord_paths[:num_shards_for_val_set]

train_ds = read_dataset(batch_size, train_tfrecord_paths)
val_ds = read_dataset(batch_size, val_tfrecord_paths, repeat=False)

train_ds_doesnt_repeat = read_dataset(batch_size, train_tfrecord_paths, repeat=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [117]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def buildModel():
    inputs = layers.Input(shape=(img_height, img_width, 3))
    
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    layer = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Dropout(0.99)(layer)

    
    outputs = layers.Dense(num_classes, activation="softmax")(layer)

    # Compile
    model = tf.keras.Model(inputs, outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy",get_f1,precision_m, recall_m]
    )
    
    return model

model = buildModel()

In [118]:
def unfreeze_model(model, num_layers):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
#     for layer in model.layers[-num_layers:]:
    for layer in model.layers:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
    return model

def freeze_model(model):
    for layer in model.layers:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
    model.layers[-1].trainable = True
    return model

In [119]:
model = unfreeze_model(model, 20)


In [120]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy",get_f1, precision_m, recall_m]
)

In [121]:
steps_per_epoch = train_ds_num_of_imgs

epochs = 10
hist = model.fit(train_ds, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=val_ds, verbose=1, batch_size=batch_size)

Epoch 1/10
472/472 [==============================] - 245s 510ms/step - loss: 7.0215 - accuracy: 0.3310 - get_f1: 0.2801 - precision_m: 0.3523 - recall_m: 0.2511 - val_loss: 2.1205 - val_accuracy: 0.0000e+00 - val_get_f1: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/10
472/472 [==============================] - 240s 509ms/step - loss: 0.6476 - accuracy: 0.9467 - get_f1: 0.7352 - precision_m: 0.9838 - recall_m: 0.5988 - val_loss: 2.4840 - val_accuracy: 0.0000e+00 - val_get_f1: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/10
472/472 [==============================] - 240s 508ms/step - loss: 0.3224 - accuracy: 0.9981 - get_f1: 0.9693 - precision_m: 0.9994 - recall_m: 0.9415 - val_loss: 3.1489 - val_accuracy: 0.0000e+00 - val_get_f1: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/10
472/472 [==============================] - 242s 513ms/step - loss: 0.1774 - accuracy: 0.9998 - get_f1: 0.9942 - precisi

KeyboardInterrupt: 